# FluidityNonlocal: Creep Protocol with Spatial Heterogeneity

## Learning Objectives

1. **Creep Bifurcation**: Understand how applied stress above/below yield stress leads to flow vs solid-like response
2. **Spatial Effects**: Explore how fluidity diffusion creates spatial heterogeneity during creep
3. **Shear Banding**: Detect and characterize shear band formation in creep tests
4. **Local vs Nonlocal**: Compare homogeneous (local) vs heterogeneous (nonlocal) creep behavior
5. **Bayesian Analysis**: Quantify uncertainty in yield stress and diffusion coefficient

---

## Overview

Creep tests apply constant stress σ₀ and measure strain evolution γ(t):
- **Below yield**: γ(t) → γ∞ (finite strain, solid-like)
- **Above yield**: γ(t) ~ t (linear flow, liquid-like)

The **FluidityNonlocal** model includes fluidity diffusion D_f ∇²f, which:
- Smooths spatial gradients in fluidity field f(y,t)
- Can trigger shear banding even in creep (unlike local models)
- Affects the critical stress where bifurcation occurs

This notebook compares local vs nonlocal creep and demonstrates shear banding detection.

## 1. Setup

In [ ]:
# Google Colab setup (uncomment if running on Colab)
# !pip install -q rheojax jax jaxlib nlsq numpyro arviz matplotlib

import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# JAX configuration (MUST use safe_import_jax)
from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()

from rheojax.models.fluidity import FluidityNonlocal, FluidityLocal
from rheojax.core.data import RheoData
from rheojax.logging import configure_logging, get_logger

# ArviZ for Bayesian diagnostics
import arviz as az

# Configure logging
configure_logging(level="INFO")
logger = get_logger(__name__)

# Output directory
output_dir = Path("../outputs/fluidity/nonlocal/creep")
output_dir.mkdir(parents=True, exist_ok=True)

print(f"JAX version: {jax.__version__}")
print(f"JAX devices: {jax.devices()}")
print(f"Output directory: {output_dir.absolute()}")

## 2. Theory: Nonlocal Fluidity Model in Creep

### Governing Equations

**Stress evolution** (Maxwell backbone):
$$
\frac{d\sigma}{dt} = G \dot{\gamma} - \sigma f(y, t)
$$

**Fluidity evolution** (with diffusion):
$$
\frac{\partial f}{\partial t} = \frac{1}{\tau} \left[ f_0 - f + \alpha |\dot{\gamma}|^n (1 - f) \right] + D_f \nabla^2 f
$$

**Creep protocol**: σ = σ₀ (constant), solve for γ(y, t)

### Key Parameters

- **G**: Elastic modulus (Pa)
- **f₀**: Equilibrium fluidity (s⁻¹)
- **τ**: Aging timescale (s)
- **α**: Shear-rejuvenation coefficient
- **n**: Rejuvenation exponent (typically 1)
- **D_f**: Fluidity diffusion coefficient (m²/s) — **nonlocal parameter**

### Creep Regimes

1. **σ₀ < σ_y**: Delayed elastic response → plateau (solid-like)
2. **σ₀ ≈ σ_y**: Critical creep, γ ~ t^β with β < 1
3. **σ₀ > σ_y**: Steady flow, γ ~ t (liquid-like)

**Nonlocal effect**: D_f > 0 creates spatial gradients, can induce banding even below nominal yield stress.

## 3. Load Calibrated Parameters (or Use Defaults)

In [ ]:
# Try to load parameters from flow curve calibration (notebook 01)
param_file = Path("../outputs/fluidity/nonlocal/flow_curve/calibrated_params.json")

if param_file.exists():
    import json
    with open(param_file, 'r') as f:
        params = json.load(f)
    print("Loaded calibrated parameters from flow curve analysis:")
    for k, v in params.items():
        print(f"  {k}: {v:.4g}")
else:
    print("No calibrated parameters found. Using representative defaults:")
    params = {
        'G': 1000.0,          # Pa
        'f0': 0.01,           # s^-1
        'tau': 10.0,          # s
        'alpha': 2.0,         # Dimensionless
        'n': 1.0,             # Dimensionless
        'D_f': 1e-7           # m^2/s (nonlocal)
    }
    for k, v in params.items():
        print(f"  {k}: {v:.4g}")

# Estimate yield stress (approximate for fluidity model)
# For fluidity models: σ_y ~ G * f0 * tau (rough estimate)
sigma_y_est = params['G'] * params['f0'] * params['tau']
print(f"\nEstimated yield stress: σ_y ≈ {sigma_y_est:.2f} Pa")

## 4. Generate Synthetic Creep Data

In [ ]:
# Create nonlocal model for data generation
model_true = FluidityNonlocal(
    n_points=51,           # Spatial resolution
    gap_width=1e-3         # 1 mm gap
)

# Set true parameters
model_true.parameters.set_values(**params)

# Creep protocol: two stress levels
t_end = 500.0  # seconds
n_time = 200

sigma_below = 0.7 * sigma_y_est  # Below yield
sigma_above = 1.5 * sigma_y_est  # Above yield

print(f"Generating creep data at two stress levels:")
print(f"  σ_below = {sigma_below:.2f} Pa (0.7 × σ_y)")
print(f"  σ_above = {sigma_above:.2f} Pa (1.5 × σ_y)")

# Simulate creep below yield
result_below = model_true.simulate_creep(
    sigma_0=sigma_below,
    t_end=t_end,
    n_time=n_time,
    t_wait=100.0  # Pre-equilibration
)

# Simulate creep above yield
result_above = model_true.simulate_creep(
    sigma_0=sigma_above,
    t_end=t_end,
    n_time=n_time,
    t_wait=100.0
)

# Extract data
t = np.array(result_below['t'])
gamma_below = np.array(result_below['gamma_avg'])
gamma_above = np.array(result_above['gamma_avg'])

# Add measurement noise (2% strain noise)
np.random.seed(42)
noise_level = 0.02
gamma_below_noisy = gamma_below + noise_level * np.random.randn(len(gamma_below))
gamma_above_noisy = gamma_above + noise_level * np.random.randn(len(gamma_above))

print(f"\nCreep data generated:")
print(f"  Time points: {len(t)}")
print(f"  γ_below (final): {gamma_below[-1]:.4f}")
print(f"  γ_above (final): {gamma_above[-1]:.4f}")
print(f"  Noise level: {noise_level*100:.1f}% of strain")

## 5. Visualize Synthetic Creep Data

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Creep curves
ax = axes[0]
ax.plot(t, gamma_below_noisy, 'o', alpha=0.5, ms=4, label=f'σ = {sigma_below:.1f} Pa (below yield)')
ax.plot(t, gamma_above_noisy, 's', alpha=0.5, ms=4, label=f'σ = {sigma_above:.1f} Pa (above yield)')
ax.plot(t, gamma_below, '-', lw=2, color='C0', alpha=0.7, label='True (below)')
ax.plot(t, gamma_above, '-', lw=2, color='C1', alpha=0.7, label='True (above)')
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Strain γ', fontsize=12)
ax.set_title('Creep Bifurcation', fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3)

# Creep compliance J(t) = γ(t)/σ₀
ax = axes[1]
J_below = gamma_below_noisy / sigma_below
J_above = gamma_above_noisy / sigma_above
ax.loglog(t, J_below, 'o', alpha=0.5, ms=4, label='Below yield')
ax.loglog(t, J_above, 's', alpha=0.5, ms=4, label='Above yield')
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Creep Compliance J(t) (1/Pa)', fontsize=12)
ax.set_title('Creep Compliance', fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3, which='both')

plt.tight_layout()
plt.savefig(output_dir / 'synthetic_creep_data.png', dpi=150, bbox_inches='tight')
plt.show()

print("Observation: Below yield → plateau, Above yield → linear growth")

## 6. NLSQ Fitting with test_mode='creep'

In [ ]:
# Fit to above-yield creep data (more informative for parameter estimation)
model_nlsq = FluidityNonlocal(n_points=51, gap_width=1e-3)

# Create RheoData (store sigma_0 as metadata, not as x-axis)
rheo_data = RheoData(
    x=t,
    y=gamma_above_noisy,
    test_mode='creep'
)

print("Fitting nonlocal model to above-yield creep data using NLSQ...")
print(f"  Test mode: creep")
print(f"  Applied stress: σ₀ = {sigma_above:.2f} Pa")

# Note: For creep fitting, we need to pass sigma_0 explicitly
# This typically requires model-specific handling in predict()
result_nlsq = model_nlsq.fit(
    rheo_data,
    sigma_0=sigma_above,  # Pass applied stress
    max_iter=5000,
    ftol=1e-8,
    xtol=1e-8
)

print("\nNLSQ Fit Results:")
print(f"  R² = {result_nlsq.r_squared:.6f}")
print(f"  RMSE = {np.sqrt(result_nlsq.residual_variance):.6f}")
print("\nFitted parameters:")
for param_name, param in model_nlsq.parameters.items():
    true_val = params.get(param_name, np.nan)
    error = abs(param.value - true_val) / true_val * 100 if not np.isnan(true_val) else np.nan
    print(f"  {param_name}: {param.value:.4g} (true: {true_val:.4g}, error: {error:.2f}%)")

In [ ]:
# Visualize NLSQ fit
gamma_pred_nlsq = model_nlsq.predict(t, sigma_0=sigma_above, test_mode='creep')

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(t, gamma_above_noisy, 'o', alpha=0.5, ms=5, label='Synthetic data (noisy)')
ax.plot(t, gamma_above, '--', lw=2, alpha=0.7, label='True', color='gray')
ax.plot(t, gamma_pred_nlsq, '-', lw=2.5, label=f'NLSQ fit (R² = {result_nlsq.r_squared:.4f})', color='C2')
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Strain γ', fontsize=12)
ax.set_title(f'NLSQ Fit: Creep at σ = {sigma_above:.1f} Pa', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'nlsq_fit.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Bayesian Inference with NumPyro

In [ ]:
print("Running Bayesian inference (NUTS) with NLSQ warm-start...")
print(f"  num_warmup = 1000, num_samples = 2000, num_chains = 4")

# Run MCMC
posterior = model_nlsq.fit_bayesian(
    rheo_data,
    sigma_0=sigma_above,
    num_warmup=1000,
    num_samples=2000,
    num_chains=4,
    seed=42
)

print("\nBayesian inference complete.")
print(f"  Posterior samples shape: {posterior.posterior_samples['G'].shape}")

# Compute credible intervals
intervals = model_nlsq.get_credible_intervals(
    posterior.posterior_samples,
    credibility=0.95
)

print("\n95% Credible Intervals:")
for param_name in intervals.keys():
    lower, upper = intervals[param_name]
    median = np.median(posterior.posterior_samples[param_name])
    true_val = params.get(param_name, np.nan)
    in_interval = lower <= true_val <= upper if not np.isnan(true_val) else None
    print(f"  {param_name}: [{lower:.4g}, {upper:.4g}], median = {median:.4g}")
    print(f"    True value: {true_val:.4g}, In interval: {in_interval}")

## 8. ArviZ Diagnostics

In [ ]:
# Convert to ArviZ InferenceData
idata = az.from_dict(posterior.posterior_samples)

# Compute convergence diagnostics
summary = az.summary(idata, hdi_prob=0.95)
print("\nMCMC Diagnostics Summary:")
print(summary)

# Check for issues
rhat_ok = (summary['r_hat'] < 1.01).all()
ess_ok = (summary['ess_bulk'] > 400).all()

print(f"\nConvergence checks:")
print(f"  R-hat < 1.01: {rhat_ok}")
print(f"  ESS_bulk > 400: {ess_ok}")

if rhat_ok and ess_ok:
    print("  ✓ MCMC converged successfully")
else:
    print("  ⚠ MCMC may need more samples or warmup")

In [ ]:
# Trace plots
axes = az.plot_trace(
    idata,
    compact=True,
    figsize=(14, 10)
)
plt.suptitle('MCMC Trace Plots', fontsize=14, fontweight='bold', y=1.001)
plt.tight_layout()
plt.savefig(output_dir / 'trace_plots.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Pair plot (correlations)
axes = az.plot_pair(
    idata,
    kind='kde',
    marginals=True,
    figsize=(12, 12)
)
plt.suptitle('Posterior Correlations', fontsize=14, fontweight='bold', y=1.001)
plt.tight_layout()
plt.savefig(output_dir / 'pair_plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("Strong correlations indicate parameter coupling (common in fluidity models)")

In [ ]:
# Forest plot (credible intervals)
ax = az.plot_forest(
    idata,
    combined=True,
    hdi_prob=0.95,
    figsize=(10, 6)
)
plt.title('95% Credible Intervals', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig(output_dir / 'forest_plot.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Compare Local vs Nonlocal Creep Response

In [ ]:
# Create local model (D_f = 0) with same parameters
model_local = FluidityLocal()
local_params = params.copy()
local_params.pop('D_f', None)  # Remove D_f (not in local model)
model_local.parameters.set_values(**local_params)

# Simulate local creep
result_local_below = model_local.simulate_creep(
    sigma_0=sigma_below,
    t_end=t_end,
    n_time=n_time
)

result_local_above = model_local.simulate_creep(
    sigma_0=sigma_above,
    t_end=t_end,
    n_time=n_time
)

# Compare local vs nonlocal
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Below yield
ax = axes[0]
ax.plot(t, result_local_below['gamma'], '-', lw=2.5, label='Local (homogeneous)', color='C0')
ax.plot(t, gamma_below, '--', lw=2.5, label='Nonlocal (heterogeneous)', color='C1')
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Strain γ', fontsize=12)
ax.set_title(f'Creep Below Yield (σ = {sigma_below:.1f} Pa)', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)

# Above yield
ax = axes[1]
ax.plot(t, result_local_above['gamma'], '-', lw=2.5, label='Local', color='C0')
ax.plot(t, gamma_above, '--', lw=2.5, label='Nonlocal', color='C1')
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Strain γ', fontsize=12)
ax.set_title(f'Creep Above Yield (σ = {sigma_above:.1f} Pa)', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'local_vs_nonlocal.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nKey differences:")
print("  - Local model: Homogeneous response, sharp yield transition")
print("  - Nonlocal model: Spatial heterogeneity, smoother transition due to diffusion")

## 10. Shear Banding Detection During Creep

In [ ]:
# Analyze spatial profiles at final time for both stress levels
y_coords = result_above['y']  # Spatial coordinates

# Extract final state
gamma_dot_below_final = np.array(result_below['gamma_dot'][-1, :])  # Shape: (n_points,)
gamma_dot_above_final = np.array(result_above['gamma_dot'][-1, :])  # Shape: (n_points,)
f_below_final = np.array(result_below['f'][-1, :])  # Shape: (n_points,)
f_above_final = np.array(result_above['f'][-1, :])  # Shape: (n_points,)

# Shear banding detection (simple threshold)
banding_threshold = 0.1  # 10% variation

cv_below = np.std(gamma_dot_below_final) / (np.mean(gamma_dot_below_final) + 1e-10)
cv_above = np.std(gamma_dot_above_final) / (np.mean(gamma_dot_above_final) + 1e-10)

has_banding_below = cv_below > banding_threshold
has_banding_above = cv_above > banding_threshold

print(f"Shear banding detection at t = {t_end:.0f} s:")
print(f"  Below yield (σ = {sigma_below:.1f} Pa):")
print(f"    CV(γ̇) = {cv_below:.4f}, Banding: {has_banding_below}")
print(f"  Above yield (σ = {sigma_above:.1f} Pa):")
print(f"    CV(γ̇) = {cv_above:.4f}, Banding: {has_banding_above}")

In [ ]:
# Visualize spatial profiles
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Shear rate profiles
ax = axes[0, 0]
ax.plot(y_coords * 1e3, gamma_dot_below_final, '-o', lw=2, ms=4, label='Below yield')
ax.plot(y_coords * 1e3, gamma_dot_above_final, '-s', lw=2, ms=4, label='Above yield')
ax.set_xlabel('Position y (mm)', fontsize=12)
ax.set_ylabel('Shear rate γ̇ (s⁻¹)', fontsize=12)
ax.set_title('Spatial Shear Rate Profile', fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3)

# Fluidity profiles
ax = axes[0, 1]
ax.plot(y_coords * 1e3, f_below_final, '-o', lw=2, ms=4, label='Below yield')
ax.plot(y_coords * 1e3, f_above_final, '-s', lw=2, ms=4, label='Above yield')
ax.set_xlabel('Position y (mm)', fontsize=12)
ax.set_ylabel('Fluidity f (s⁻¹)', fontsize=12)
ax.set_title('Spatial Fluidity Profile', fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3)

# Time evolution of spatial heterogeneity (above yield)
ax = axes[1, 0]
# Compute CV over time
cv_time = []
for i in range(len(result_above['t'])):
    gamma_dot_profile = np.array(result_above['gamma_dot'][i, :])
    cv = np.std(gamma_dot_profile) / (np.mean(gamma_dot_profile) + 1e-10)
    cv_time.append(cv)

ax.plot(result_above['t'], cv_time, '-', lw=2.5, color='C3')
ax.axhline(banding_threshold, ls='--', color='gray', lw=1.5, label=f'Threshold ({banding_threshold})')
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('CV(γ̇)', fontsize=12)
ax.set_title('Temporal Evolution of Spatial Heterogeneity', fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3)

# 2D spatiotemporal shear rate map
ax = axes[1, 1]
gamma_dot_2d = np.array(result_above['gamma_dot'])  # Shape: (n_time, n_points)
im = ax.contourf(
    y_coords * 1e3,
    result_above['t'],
    gamma_dot_2d,
    levels=20,
    cmap='viridis'
)
ax.set_xlabel('Position y (mm)', fontsize=12)
ax.set_ylabel('Time (s)', fontsize=12)
ax.set_title('Spatiotemporal Shear Rate Map', fontsize=13, fontweight='bold')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('γ̇ (s⁻¹)', fontsize=11)

plt.tight_layout()
plt.savefig(output_dir / 'spatial_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nSpatial analysis complete.")
if has_banding_above:
    print("  ✓ Shear banding detected in above-yield creep")
else:
    print("  No significant shear banding (may require higher stress or lower D_f)")

## 11. Save Results

In [ ]:
# Save fitted parameters
import json

fitted_params = {name: float(param.value) for name, param in model_nlsq.parameters.items()}
with open(output_dir / 'fitted_params_creep.json', 'w') as f:
    json.dump(fitted_params, f, indent=2)

# Save credible intervals
intervals_dict = {name: [float(interval[0]), float(interval[1])] for name, interval in intervals.items()}
with open(output_dir / 'credible_intervals.json', 'w') as f:
    json.dump(intervals_dict, f, indent=2)

# Save diagnostics summary
summary.to_csv(output_dir / 'mcmc_diagnostics.csv')

# Save ArviZ InferenceData
idata.to_netcdf(output_dir / 'posterior_samples.nc')

# Save synthetic data
np.savez(
    output_dir / 'synthetic_creep_data.npz',
    t=t,
    gamma_below=gamma_below_noisy,
    gamma_above=gamma_above_noisy,
    sigma_below=sigma_below,
    sigma_above=sigma_above,
    true_params=params
)

# Save spatial analysis
np.savez(
    output_dir / 'spatial_analysis.npz',
    y=y_coords,
    gamma_dot_below_final=gamma_dot_below_final,
    gamma_dot_above_final=gamma_dot_above_final,
    f_below_final=f_below_final,
    f_above_final=f_above_final,
    cv_time=cv_time,
    gamma_dot_2d=gamma_dot_2d
)

print("\nResults saved to:", output_dir.absolute())
print("  - fitted_params_creep.json")
print("  - credible_intervals.json")
print("  - mcmc_diagnostics.csv")
print("  - posterior_samples.nc")
print("  - synthetic_creep_data.npz")
print("  - spatial_analysis.npz")
print("  - *.png (5 figures)")

## 12. Key Takeaways

### Physical Insights

1. **Creep Bifurcation**: The transition from solid-like (plateau) to liquid-like (linear) response is controlled by the applied stress relative to yield stress.

2. **Nonlocal Effects**: Fluidity diffusion (D_f) creates spatial heterogeneity even in nominally homogeneous creep tests:
   - Smooths sharp yield transitions
   - Can trigger shear banding below nominal yield stress
   - Affects the critical stress for bifurcation

3. **Shear Banding in Creep**: Unlike local models, nonlocal fluidity models can exhibit spatial heterogeneity in creep:
   - Detected via coefficient of variation CV(γ̇) > threshold
   - More pronounced at intermediate stresses near yield
   - Develops over time as fluidity field evolves

4. **Local vs Nonlocal**: The key difference is spatial coupling:
   - **Local**: Homogeneous γ̇(y) = const, sharp yield
   - **Nonlocal**: Heterogeneous γ̇(y), gradual yield, banding possible

### Computational Insights

5. **NLSQ Warm-Start**: Critical for Bayesian convergence in fluidity models due to:
   - High parameter correlation (f₀, τ, α)
   - Nonlinear time evolution
   - Spatial coupling (nonlocal term)

6. **Bayesian Uncertainty**: Creep data provides strong constraints on:
   - Yield stress τ_y ~ G·f₀·τ (from bifurcation point)
   - Diffusion coefficient D_f (from spatial heterogeneity)
   - Weaker constraints on n, α (need multiple stress levels)

7. **Diagnostics**: Always check:
   - **R-hat < 1.01**: Chain convergence
   - **ESS > 400**: Effective sample size
   - **Pair plots**: Parameter correlations
   - **Spatial CV**: Shear banding detection

### Experimental Design

8. **Stress Sweep**: To fully characterize creep response:
   - Test at 5-7 stress levels: 0.5σ_y, 0.7σ_y, 0.9σ_y, σ_y, 1.2σ_y, 1.5σ_y, 2σ_y
   - Look for delayed yielding near σ_y (thixotropic signature)
   - Monitor spatial profiles if possible (velocimetry)

9. **Time Window**: Ensure sufficient observation time:
   - Below yield: t_end > 10τ to reach plateau
   - Above yield: t_end > 5τ to achieve steady flow
   - Pre-equilibration (t_wait) essential for reproducibility

10. **Spatial Resolution**: For nonlocal models:
    - n_points ≥ 50 for accurate spatial gradients
    - Gap width H affects D_f scaling: D_f ~ H² for dimensionless analysis
    - Use velocimetry (PIV, NMR) to validate spatial predictions

---

**Next Steps:**
- Notebook 10: LAOS (Large Amplitude Oscillatory Shear) with nonlinear rheology
- Notebook 11: Model comparison (local vs nonlocal, different coupling modes)
- Notebook 12: Experimental validation with real data (e.g., carbopol, hair gel)